In [6]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Load Dataset

In [3]:


# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])

# Load datasets
train_dataset = datasets.ImageFolder(
    root="../data/binary_classification/train",
    transform=transform
)

val_dataset = datasets.ImageFolder(
    root="../data/binary_classification/val",
    transform=transform
)

test_dataset = datasets.ImageFolder(
    root="../data/binary_classification/test",
    transform=transform
)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [5]:
print(train_dataset.class_to_idx)

{'Cat': 0, 'Dog': 1}


# Minimal CNN Architecture (PyTorch)

In [7]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        # Feature Extraction
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        # Classification
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(8 * 32 * 32, 2)  # 2 classes: Cat, Dog
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [10]:
model = SimpleCNN()
print(model)

SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=8192, out_features=2, bias=True)
  )
)


In [11]:
criterion = nn.CrossEntropyLoss()

In [12]:
import torch.optim as optim
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [13]:
device = torch.device("cpu")
model = model.to(device)

In [15]:
num_epochs = 15  # small for CPU

for epoch in range(num_epochs):
    model.train()  # training mode
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Step 1: Forward pass
        outputs = model(images)

        # Step 2: Compute loss
        loss = criterion(outputs, labels)

        # Step 3: Backpropagation
        optimizer.zero_grad()
        loss.backward()

        # Step 4: Update weights
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")


Epoch [1/15], Loss: 0.4890
Epoch [2/15], Loss: 0.4574
Epoch [3/15], Loss: 0.4425
Epoch [4/15], Loss: 0.4310
Epoch [5/15], Loss: 0.3986
Epoch [6/15], Loss: 0.3657
Epoch [7/15], Loss: 0.3552
Epoch [8/15], Loss: 0.3291
Epoch [9/15], Loss: 0.2987
Epoch [10/15], Loss: 0.3026
Epoch [11/15], Loss: 0.2796
Epoch [12/15], Loss: 0.2693
Epoch [13/15], Loss: 0.2473
Epoch [14/15], Loss: 0.2449
Epoch [15/15], Loss: 0.2245


In [16]:
num_epochs = 15  # small for CPU

for epoch in range(num_epochs):
    model.train()  # training mode
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Step 1: Forward pass
        outputs = model(images)

        # Step 2: Compute loss
        loss = criterion(outputs, labels)

        # Step 3: Backpropagation
        optimizer.zero_grad()
        loss.backward()

        # Step 4: Update weights
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")


Epoch [1/15], Loss: 0.2019
Epoch [2/15], Loss: 0.1954
Epoch [3/15], Loss: 0.1777
Epoch [4/15], Loss: 0.1659
Epoch [5/15], Loss: 0.1599
Epoch [6/15], Loss: 0.1534
Epoch [7/15], Loss: 0.1460
Epoch [8/15], Loss: 0.1374
Epoch [9/15], Loss: 0.1214
Epoch [10/15], Loss: 0.1231
Epoch [11/15], Loss: 0.1200
Epoch [12/15], Loss: 0.1171
Epoch [13/15], Loss: 0.1086
Epoch [14/15], Loss: 0.1021
Epoch [15/15], Loss: 0.0941


In [17]:
model.eval()  # evaluation mode

correct = 0
total = 0
val_loss = 0.0

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

avg_val_loss = val_loss / len(val_loader)
val_accuracy = 100 * correct / total

print(f"Validation Loss: {avg_val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy:.2f}%")


Validation Loss: 0.8240
Validation Accuracy: 66.00%


In [18]:
model.eval()  # evaluation mode

correct = 0
total = 0
test_loss = 0.0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

avg_test_loss = test_loss / len(test_loader)
test_accuracy = 100 * correct / total

print(f"Test Loss: {avg_test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.2f}%")


Test Loss: 1.0519
Test Accuracy: 60.00%
